<a href="https://colab.research.google.com/github/daystram/ml-playground/blob/master/02_gym_cartpole_keras/02_gym_cartpole_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -r ml-playground > /dev/null 2>&1
!git clone https://github.com/daystram/ml-playground.git
!sh ml-playground/helper/setup.sh > /dev/null 2>&1

Cloning into 'ml-playground'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 112 (delta 63), reused 68 (delta 29), pack-reused 0
Receiving objects: 100% (112/112), 16.36 KiB | 5.45 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [0]:
import sys
sys.path.append('/content/ml-playground')

In [3]:
import gym

env = gym.make("CartPole-v1")
print(env.observation_space)
print(env.action_space)

Box(4,)
Discrete(2)


In [4]:
import numpy as np

acc_fitness = []
training_data = []

sample_count = 200000
steps = 1000
goal_tres = 120

for _ in range(sample_count):
  fitness = 0
  action_seq = []
  prev_observation = env.reset()
  
  for _ in range(steps):
    action = np.random.randint(0, 2)
    action_seq.append([prev_observation, action])
    prev_observation, reward, done, _ = env.step(action)
    fitness += reward
    if done: break
    
  if fitness >= goal_tres:
    acc_fitness.append(fitness)
    for seq in action_seq:
      if seq[1] == 1:
        action = [0, 1]
      else:
        action = [1, 0]
      training_data.append([seq[0], action])
env.close()
print("{} samples".format(len(training_data)))

2172 samples


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

drop_rate = 0.25
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(4,)))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(512, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(2, activation='relu'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])

model.fit(X, y, epochs=20, batch_size=64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
2172/2172 [==============================] - 2s 718us/step - loss: 1.0437 - acc: 0.5401
Epoch 2/20
2172/2172 [==============================] - 0s 79us/step - loss: 0.6739 - acc: 0.5852
Epoch 3/20
2172/2172 [==============================] - 0s 73us/step - loss: 0.6666 - acc: 0.5999
Epoch 4/20
2172/2172 [==============================] - 0s 79us/step - loss: 0.6681 - acc: 0.5944
Epoch 5/20
2172/2172 [==============================] - 0s 75us/step - loss: 0.6680 - acc: 0.6004
Epoch 6/20
2172/2172 [==============================] - 0s 76us/step - loss: 0.6680 - acc: 0.5958
Epoch 7/20
2172/2172 [==============================] - 0s 77us/step - loss: 0.6607 - acc: 0.6091
Epoch 8/20
2172/2172 [==============================] - 0s 81us/step - loss: 0.6538 - acc: 0.6179
Epoch 9/20
2172/2172 [==============================] - 0s 76us/step - loss: 0.6573 - acc: 0.6174
Epoch 10/20
2172/2172 [==============================] - 0s 76us/step

In [7]:
from helper import nbdisplay as disp

!rm -r video
env = disp.wrap_env(gym.make("CartPole-v1"))
observation = env.reset()
fitness = 0
print(np.array([observation]))
for _ in range(steps):
  action = np.argmax(model.predict(np.array([observation])))
  observation, reward, done, _ = env.step(action)
  fitness += reward
  if done: break

env.close()
disp.show_video()

print("------ Agent: Reward {}".format(fitness))  

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


[[-0.03022734 -0.01468179  0.0070167  -0.00241632]]


------ Agent: Reward 500.0
